### This is the Notebook for Owen Monroe's Homework Assigment 2 for IS-567 Text Mining
##### September 23, 2023

1. Loading the Dataset

In [376]:
import pandas as pd
import numpy as np
import nltk
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack



In [377]:
train_df = pd.read_csv('chatgpt_train.csv')
test_df = pd.read_csv('chatgpt_test.csv')
train_df['review'][24]

nan

In [378]:
# Creates lists and reporting length

tr_rev_list = train_df['review'].to_list()
te_rev_list = test_df['review'].to_list()
tr_rev_list
te_rev_list
print(len(tr_rev_list))
print(len(te_rev_list))


1834
458


In [379]:
# Removing Empties and reporting length
full_train_df = train_df.dropna()
full_test_df = test_df.dropna()

print(len(full_train_df['review'].to_list()))
print(len(full_test_df['review'].to_list()))

1829
458


2. POS Tagging

In [380]:
# Copying training Data and POS tagging
train_task2_df = full_train_df

# Tokenizing the Reviews
train_task2_df['NLTK Tokenized Reviews'] = train_task2_df['review'].apply(nltk.word_tokenize)


/var/folders/8c/38d5fhqx2_jf385d_b3cq3j00000gn/T/ipykernel_45660/3634727750.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_task2_df['NLTK Tokenized Reviews'] = train_task2_df['review'].apply(nltk.word_tokenize)


In [381]:
# Library Download

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/owenmonroe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/owenmonroe/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [382]:
# POS Tagging and Reporting

train_task2_df['POS Tagged Tokens'] = train_task2_df['NLTK Tokenized Reviews'].apply(pos_tag)
train_task2_df[['review','POS Tagged Tokens']][20:23]

/var/folders/8c/38d5fhqx2_jf385d_b3cq3j00000gn/T/ipykernel_45660/121813578.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_task2_df['POS Tagged Tokens'] = train_task2_df['NLTK Tokenized Reviews'].apply(pos_tag)


,review,POS Tagged Tokens
21,ChatGPT is an impressive AI language model tha...,"[(ChatGPT, NNP), (is, VBZ), (an, DT), (impress..."
22,Though there is a problem that I had found mys...,"[(Though, IN), (there, EX), (is, VBZ), (a, DT)..."
23,"I downloaded the ChatGPT app with high hopes, ...","[(I, PRP), (downloaded, VBD), (the, DT), (Chat..."


3. Extract Unigram Features

In [383]:
# Building Model
train_text = full_train_df["review"]
test_text = full_test_df["review"]

# Setting n-gram to uni-grams
vectorizer = CountVectorizer(ngram_range = (1,1))

# Creating training data representation
train_data_cv = vectorizer.fit_transform(train_text.values.astype('U'))
print(train_data_cv.shape,"\n") 

# Creating test data representation
test_data_cv = vectorizer.transform(test_text.values.astype('U'))
print(test_data_cv.shape,"\n") 


(1829, 5551) 

(458, 5551) 



In [384]:
type(train_data_cv)


scipy.sparse._csr.csr_matrix

In [385]:
print(test_data_cv)

  (0, 694)	1
  (0, 2885)	1
  (1, 2233)	1
  (1, 2278)	1
  (1, 3129)	1
  (1, 4898)	1
  (2, 1929)	1
  (2, 2666)	1
  (3, 2645)	1
  (3, 3193)	1
  (3, 5384)	1
  (4, 2170)	1
  (5, 291)	1
  (5, 2135)	1
  (5, 2660)	1
  (5, 3891)	1
  (5, 4932)	1
  (6, 1923)	1
  (7, 291)	1
  (7, 3311)	1
  (8, 2146)	2
  (8, 2660)	1
  (8, 3372)	1
  (8, 4932)	1
  (10, 776)	1
  :	:
  (455, 5437)	1
  (455, 5464)	2
  (455, 5466)	1
  (455, 5503)	5
  (456, 115)	1
  (456, 197)	1
  (456, 230)	1
  (456, 373)	1
  (456, 386)	1
  (456, 388)	1
  (456, 497)	1
  (456, 562)	1
  (456, 701)	1
  (456, 776)	1
  (456, 1435)	1
  (456, 1974)	1
  (456, 2059)	1
  (456, 3303)	1
  (456, 3341)	2
  (456, 4216)	1
  (456, 4507)	1
  (456, 4906)	1
  (456, 4985)	1
  (456, 5403)	1
  (456, 5464)	1


4. Train and Evaluate Classifier

In [386]:
# Define true labels from train set

x_train_cv = train_data_cv
y_train_rev_series = full_train_df["rating"]
x_test_cv = test_data_cv
y_test_rev_series = full_test_df["rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_cv, y_train_rev_series)

# Predict the labels for the test data

predictions = mnb_model.predict(x_test_cv)
predictions

array([1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 3, 1, 1, 5, 5, 5, 1, 1, 5, 5, 5,
       1, 5, 5, 1, 5, 5, 1, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 1, 3, 5, 5,
       5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 1, 5, 3, 4, 5, 5, 1, 5, 5, 1, 1, 5,
       5, 5, 5, 1, 5, 4, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 5, 5,
       5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 1, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5,
       1, 1, 1, 5, 1, 1, 5, 5, 5, 5, 5, 2, 2, 5, 3, 1, 5, 5, 5, 1, 5, 5,
       5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 2, 1, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,
       5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 1, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 1, 3, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 5, 5, 5, 5,
       1, 4, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 1, 5,

In [387]:
# Evaluating the Classifier
# Accuracy Classification Score

true_label_list = y_test_rev_series.to_list()
predict_label_list = list(predictions)

acc_score = accuracy_score(true_label_list, predict_label_list)
print(acc_score)


0.6331877729257642


In [388]:
# Precision Score
from sklearn.metrics import precision_score

true_label_array = np.array(true_label_list)
predict_label_array = np.array(predict_label_list)

prec_score_macro = precision_score(true_label_array, predict_label_array, average='macro')
prec_score_micro = precision_score(true_label_array, predict_label_array, average='micro')
prec_score_weighted = precision_score(true_label_array, predict_label_array, average='weighted')

print(f'macro precision score = {prec_score_macro}')
print(f'micro precision score = {prec_score_micro}')
print(f'weighted precision score = {prec_score_weighted}')
prec_none_array = precision_score(true_label_array, predict_label_array, average=None)
prec_none_array


macro precision score = 0.4365165675446049
micro precision score = 0.6331877729257642
weighted precision score = 0.5855492442277733


array([0.56565657, 0.4       , 0.13333333, 0.38888889, 0.69470405])

In [389]:
# Recall Score
from sklearn.metrics import recall_score

rec_score_macro = recall_score(true_label_array, predict_label_array, average='macro')
rec_score_micro = recall_score(true_label_array, predict_label_array, average='micro')
rec_score_weighted = recall_score(true_label_array, predict_label_array, average='weighted')

print(f'macro recall score = {prec_score_macro}')
print(f'micro recall score = {prec_score_micro}')
print(f'weighted recall score = {prec_score_weighted}')
rec_none_array = recall_score(true_label_array, predict_label_array, average=None)
rec_none_array


macro recall score = 0.4365165675446049
micro recall score = 0.6331877729257642
weighted recall score = 0.5855492442277733


array([0.48695652, 0.07692308, 0.08      , 0.15909091, 0.89919355])

In [390]:
# F1 Score

from sklearn.metrics import f1_score
f1_score_macro = f1_score(true_label_array, predict_label_array, average='macro')
f1_score_micro = f1_score(true_label_array, predict_label_array, average='micro')
f1_score_weighted = f1_score(true_label_array, predict_label_array, average='weighted')

print(f'macro F1 score = {f1_score_macro}')
print(f'micro F1 score = {f1_score_micro}')
print(f'weighted F1 score = {f1_score_weighted}')
f1_none_array = f1_score(true_label_array, predict_label_array, average=None)
f1_none_array

macro F1 score = 0.35240689572225525
micro F1 score = 0.6331877729257642
weighted F1 score = 0.5903218267264796


array([0.52336449, 0.12903226, 0.1       , 0.22580645, 0.78383128])

In [391]:
# Evaluating Labels: 1,  Unigrams (Chat-GPT Help)

# Constructing Filter for Label == 1
label_1 = 1 

label_filter = [1 if label == label_1 else 0 for label in true_label_list]

true_labels_for_label_1 = [true_label_list[i] for i, mask in enumerate(label_filter) if mask == 1]
predicted_labels_for_lable_1 = [predict_label_list[i] for i, mask in enumerate(label_filter) if mask == 1]

# Accuracy Scoring for New List
accuracy_for_label_1 = accuracy_score(true_labels_for_label_1, predicted_labels_for_lable_1)

# Precision Scoring for New List
prec_score_macro_label_1 = precision_score(true_labels_for_label_1, predicted_labels_for_lable_1, average='macro')
prec_score_micro_label_1 = precision_score(true_labels_for_label_1, predicted_labels_for_lable_1, average='micro')
prec_score_weighted_label_1 = precision_score(true_labels_for_label_1, predicted_labels_for_lable_1, average='weighted')

# Recall Scoring for New List
rec_score_macro_label_1 = recall_score(true_labels_for_label_1, predicted_labels_for_lable_1, average='macro')
rec_score_micro_label_1 = recall_score(true_labels_for_label_1, predicted_labels_for_lable_1, average='micro')
rec_score_weighted_label_1 = recall_score(true_labels_for_label_1, predicted_labels_for_lable_1, average='weighted')

#F1 Scoring for New List
f1_score_macro_label_1 = f1_score(true_labels_for_label_1, predicted_labels_for_lable_1, average='macro')
f1_score_micro_label_1 = f1_score(true_labels_for_label_1, predicted_labels_for_lable_1, average='micro')
f1_score_weighted_label_1 = f1_score(true_labels_for_label_1, predicted_labels_for_lable_1, average='weighted')


# Reporting All 
print('Reporting Scores for Label 1')
print()
print(f'Accuracy for label {label_1}: {accuracy_for_label_1:.2f}')
print()
print(f'macro precision score for label {label_1} = {prec_score_macro_label_1}')
print(f'micro precision score for label {label_1} = {prec_score_micro_label_1}')
print(f'weighted precision score for label {label_1} = {prec_score_weighted_label_1}')
print()
print(f'macro recall score for label {label_1} = {rec_score_macro_label_1}')
print(f'micro recall score for label {label_1} = {rec_score_micro_label_1}')
print(f'weighted recall score for label {label_1} = {rec_score_weighted_label_1}')
print()
print(f'macro F1 score for label {label_1} = {f1_score_macro_label_1}')
print(f'micro F1 score for label {label_1} = {f1_score_micro_label_1}')
print(f'weighted F1 score for label {label_1} = {f1_score_weighted_label_1}')


Reporting Scores for Label 1

Accuracy for label 1: 0.49

macro precision score for label 1 = 0.25
micro precision score for label 1 = 0.48695652173913045
weighted precision score for label 1 = 1.0

macro recall score for label 1 = 0.12173913043478261
micro recall score for label 1 = 0.48695652173913045
weighted recall score for label 1 = 0.48695652173913045

macro F1 score for label 1 = 0.16374269005847952
micro F1 score for label 1 = 0.48695652173913045
weighted F1 score for label 1 = 0.6549707602339181


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [392]:
# Evaluating Labels: 2, Unigrams (Chat-GPT Help)

# Constructing Filter for Label == 2
label_2 = 2 

label_filter = [2 if label == label_2 else 0 for label in true_label_list]

true_labels_for_label_2 = [true_label_list[i] for i, mask in enumerate(label_filter) if mask == 2]
predicted_labels_for_lable_2 = [predict_label_list[i] for i, mask in enumerate(label_filter) if mask == 2]

# Accuracy Scoring for New List
accuracy_for_label_2 = accuracy_score(true_labels_for_label_2, predicted_labels_for_lable_2)

# Precision Scoring for New List
prec_score_macro_label_2 = precision_score(true_labels_for_label_2, predicted_labels_for_lable_2, average='macro')
prec_score_micro_label_2 = precision_score(true_labels_for_label_2, predicted_labels_for_lable_2, average='micro')
prec_score_weighted_label_2 = precision_score(true_labels_for_label_2, predicted_labels_for_lable_2, average='weighted')

# Recall Scoring for New List
rec_score_macro_label_2 = recall_score(true_labels_for_label_2, predicted_labels_for_lable_2, average='macro')
rec_score_micro_label_2 = recall_score(true_labels_for_label_2, predicted_labels_for_lable_2, average='micro')
rec_score_weighted_label_2 = recall_score(true_labels_for_label_2, predicted_labels_for_lable_2, average='weighted')

#F1 Scoring for New List
f1_score_macro_label_2 = f1_score(true_labels_for_label_2, predicted_labels_for_lable_2, average='macro')
f1_score_micro_label_2 = f1_score(true_labels_for_label_2, predicted_labels_for_lable_2, average='micro')
f1_score_weighted_label_2 = f1_score(true_labels_for_label_2, predicted_labels_for_lable_2, average='weighted')


# Reporting All 
print('Reporting Scores for Label 2')
print()
print(f'Accuracy for label {label_2}: {accuracy_for_label_2:.2f}')
print()
print(f'macro precision score for label {label_2} = {prec_score_macro_label_2}')
print(f'micro precision score for label {label_2} = {prec_score_micro_label_2}')
print(f'weighted precision score for label {label_2} = {prec_score_weighted_label_2}')
print()
print(f'macro recall score for label {label_2} = {rec_score_macro}')
print(f'micro recall score for label {label_2} = {rec_score_micro}')
print(f'weighted recall score for label {label_2} = {rec_score_weighted}')
print()
print(f'macro F1 score for label {label_2} = {f1_score_macro_label_2}')
print(f'micro F1 score for label {label_2} = {f1_score_micro_label_2}')
print(f'weighted F1 score for label {label_2} = {f1_score_weighted_label_2}')


Reporting Scores for Label 2

Accuracy for label 2: 0.08

macro precision score for label 2 = 0.2
micro precision score for label 2 = 0.07692307692307693
weighted precision score for label 2 = 1.0

macro recall score for label 2 = 0.34043281122804264
micro recall score for label 2 = 0.6331877729257642
weighted recall score for label 2 = 0.6331877729257642

macro F1 score for label 2 = 0.028571428571428574
micro F1 score for label 2 = 0.07692307692307693
weighted F1 score for label 2 = 0.14285714285714288


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [393]:
# Evaluating Labels: 3, Unigrams (Chat-GPT Help)

# Constructing Filter for Label == 3
label_3 = 3 

label_filter = [3 if label == label_2 else 0 for label in true_label_list]

true_labels_for_label_3 = [true_label_list[i] for i, mask in enumerate(label_filter) if mask == 3]
predicted_labels_for_lable_3 = [predict_label_list[i] for i, mask in enumerate(label_filter) if mask == 3]

# Accuracy Scoring for New List
accuracy_for_label_3 = accuracy_score(true_labels_for_label_3, predicted_labels_for_lable_3)

# Precision Scoring for New List
prec_score_macro_label_3 = precision_score(true_labels_for_label_3, predicted_labels_for_lable_3, average='macro')
prec_score_micro_label_3 = precision_score(true_labels_for_label_3, predicted_labels_for_lable_3, average='micro')
prec_score_weighted_label_3 = precision_score(true_labels_for_label_3, predicted_labels_for_lable_3, average='weighted')

# Recall Scoring for New List
rec_score_macro_label_3 = recall_score(true_labels_for_label_3, predicted_labels_for_lable_3, average='macro')
rec_score_micro_label_3 = recall_score(true_labels_for_label_3, predicted_labels_for_lable_3, average='micro')
rec_score_weighted_label_3 = recall_score(true_labels_for_label_3, predicted_labels_for_lable_3, average='weighted')

#F1 Scoring for New List
f1_score_macro_label_3 = f1_score(true_labels_for_label_3, predicted_labels_for_lable_3, average='macro')
f1_score_micro_label_3 = f1_score(true_labels_for_label_3, predicted_labels_for_lable_3, average='micro')
f1_score_weighted_label_3 = f1_score(true_labels_for_label_3, predicted_labels_for_lable_3, average='weighted')


# Reporting All 
print('Reporting Scores for Label 3')
print()
print(f'Accuracy for label {label_3}: {accuracy_for_label_3:.2f}')
print()
print(f'macro precision score for label {label_3} = {prec_score_macro_label_3}')
print(f'micro precision score for label {label_3} = {prec_score_micro_label_3}')
print(f'weighted precision score for label {label_3} = {prec_score_weighted_label_3}')
print()
print(f'macro recall score for label {label_3} = {rec_score_macro_label_3}')
print(f'micro recall score for label {label_3} = {rec_score_micro_label_3}')
print(f'weighted recall score for label {label_3} = {rec_score_weighted_label_3}')
print()
print(f'macro F1 score for label {label_3} = {f1_score_macro_label_3}')
print(f'micro F1 score for label {label_3} = {f1_score_micro_label_3}')
print(f'weighted F1 score for label {label_3} = {f1_score_weighted_label_3}')

Reporting Scores for Label 3

Accuracy for label 3: 0.08

macro precision score for label 3 = 0.2
micro precision score for label 3 = 0.07692307692307693
weighted precision score for label 3 = 1.0

macro recall score for label 3 = 0.015384615384615385
micro recall score for label 3 = 0.07692307692307693
weighted recall score for label 3 = 0.07692307692307693

macro F1 score for label 3 = 0.028571428571428574
micro F1 score for label 3 = 0.07692307692307693
weighted F1 score for label 3 = 0.14285714285714288


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [394]:
# Evaluating Labels: 4, Unigrams (Chat-GPT Help)

# Constructing Filter for Label == 4
label_4 = 4 

label_filter = [4 if label == label_2 else 0 for label in true_label_list]

true_labels_for_label_4 = [true_label_list[i] for i, mask in enumerate(label_filter) if mask == 4]
predicted_labels_for_lable_4 = [predict_label_list[i] for i, mask in enumerate(label_filter) if mask == 4]

# Accuracy Scoring for New List
accuracy_for_label_4 = accuracy_score(true_labels_for_label_4, predicted_labels_for_lable_4)

# Precision Scoring for New List
prec_score_macro_label_4 = precision_score(true_labels_for_label_4, predicted_labels_for_lable_4, average='macro')
prec_score_micro_label_4 = precision_score(true_labels_for_label_4, predicted_labels_for_lable_4, average='micro')
prec_score_weighted_label_4 = precision_score(true_labels_for_label_4, predicted_labels_for_lable_4, average='weighted')

# Recall Scoring for New List
rec_score_macro_label_4 = recall_score(true_labels_for_label_4, predicted_labels_for_lable_4, average='macro')
rec_score_micro_label_4 = recall_score(true_labels_for_label_4, predicted_labels_for_lable_4, average='micro')
rec_score_weighted_label_4 = recall_score(true_labels_for_label_4, predicted_labels_for_lable_4, average='weighted')

#F1 Scoring for New List
f1_score_macro_label_4 = f1_score(true_labels_for_label_4, predicted_labels_for_lable_4, average='macro')
f1_score_micro_label_4 = f1_score(true_labels_for_label_4, predicted_labels_for_lable_4, average='micro')
f1_score_weighted_label_4 = f1_score(true_labels_for_label_4, predicted_labels_for_lable_4, average='weighted')


# Reporting All 
print('Reporting Scores for Label 4')
print()
print(f'Accuracy for label {label_4}: {accuracy_for_label_4:.2f}')
print()
print(f'macro precision score for label {label_4} = {prec_score_macro_label_4}')
print(f'micro precision score for label {label_4} = {prec_score_micro_label_4}')
print(f'weighted precision score for label {label_4} = {prec_score_weighted_label_4}')
print()
print(f'macro recall score for label {label_4} = {rec_score_macro_label_4}')
print(f'micro recall score for label {label_4} = {rec_score_micro_label_4}')
print(f'weighted recall score for label {label_4} = {rec_score_weighted_label_4}')
print()
print(f'macro F1 score for label {label_4} = {f1_score_macro_label_4}')
print(f'micro F1 score for label {label_4} = {f1_score_micro_label_4}')
print(f'weighted F1 score for label {label_4} = {f1_score_weighted_label_4}')

Reporting Scores for Label 4

Accuracy for label 4: 0.08

macro precision score for label 4 = 0.2
micro precision score for label 4 = 0.07692307692307693
weighted precision score for label 4 = 1.0

macro recall score for label 4 = 0.015384615384615385
micro recall score for label 4 = 0.07692307692307693
weighted recall score for label 4 = 0.07692307692307693

macro F1 score for label 4 = 0.028571428571428574
micro F1 score for label 4 = 0.07692307692307693
weighted F1 score for label 4 = 0.14285714285714288


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [395]:
# Evaluating Labels: 5, Unigrams (Chat-GPT Help)

# Constructing Filter for Label == 5
label_5 = 5

label_filter = [5 if label == label_2 else 0 for label in true_label_list]

true_labels_for_label_5 = [true_label_list[i] for i, mask in enumerate(label_filter) if mask == 5]
predicted_labels_for_lable_5 = [predict_label_list[i] for i, mask in enumerate(label_filter) if mask == 5]

# Accuracy Scoring for New List
accuracy_for_label_5 = accuracy_score(true_labels_for_label_5, predicted_labels_for_lable_5)

# Precision Scoring for New List
prec_score_macro_label_5 = precision_score(true_labels_for_label_5, predicted_labels_for_lable_5, average='macro')
prec_score_micro_label_5 = precision_score(true_labels_for_label_5, predicted_labels_for_lable_5, average='micro')
prec_score_weighted_label_5 = precision_score(true_labels_for_label_5, predicted_labels_for_lable_5, average='weighted')

# Recall Scoring for New List
rec_score_macro_label_5 = recall_score(true_labels_for_label_5, predicted_labels_for_lable_5, average='macro')
rec_score_micro_label_5 = recall_score(true_labels_for_label_5, predicted_labels_for_lable_5, average='micro')
rec_score_weighted_label_5 = recall_score(true_labels_for_label_5, predicted_labels_for_lable_5, average='weighted')

#F1 Scoring for New List
f1_score_macro_label_5= f1_score(true_labels_for_label_5, predicted_labels_for_lable_5, average='macro')
f1_score_micro_label_5 = f1_score(true_labels_for_label_5, predicted_labels_for_lable_5, average='micro')
f1_score_weighted_label_5 = f1_score(true_labels_for_label_5, predicted_labels_for_lable_5, average='weighted')


# Reporting All 
print('Reporting Scores for Label 5')
print()
print(f'Accuracy for label {label_5}: {accuracy_for_label_4:.2f}')
print()
print(f'macro precision score for label {label_5} = {prec_score_macro_label_5}')
print(f'micro precision score for label {label_5} = {prec_score_micro_label_5}')
print(f'weighted precision score for label {label_5} = {prec_score_weighted_label_5}')
print()
print(f'macro recall score for label {label_5} = {rec_score_macro_label_5}')
print(f'micro recall score for label {label_5} = {rec_score_micro_label_5}')
print(f'weighted recall score for label {label_5} = {rec_score_weighted_label_5}')
print()
print(f'macro F1 score for label {label_5} = {f1_score_macro_label_5}')
print(f'micro F1 score for label {label_5} = {f1_score_micro_label_5}')
print(f'weighted F1 score for label {label_5} = {f1_score_weighted_label_5}')

Reporting Scores for Label 5

Accuracy for label 5: 0.08

macro precision score for label 5 = 0.2
micro precision score for label 5 = 0.07692307692307693
weighted precision score for label 5 = 1.0

macro recall score for label 5 = 0.015384615384615385
micro recall score for label 5 = 0.07692307692307693
weighted recall score for label 5 = 0.07692307692307693

macro F1 score for label 5 = 0.028571428571428574
micro F1 score for label 5 = 0.07692307692307693
weighted F1 score for label 5 = 0.14285714285714288


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5. Add Bi-grams Features (Continue to Evaluate)

In [396]:
# Extracting Bi-gram features 

# Building Model
train_text = full_train_df["review"]
test_text = full_test_df["review"]

# Setting n-gram range to uni-grams and bigrams
vectorizer = CountVectorizer(ngram_range = (1, 2))

# Creating training data representation
train_data_bigram_cv = vectorizer.fit_transform(train_text.values.astype('U'))
print(train_data_bigram_cv.shape,"\n") 

# Creating test data representation
test_data_bigram_cv = vectorizer.transform(test_text.values.astype('U'))
print(test_data_bigram_cv.shape,"\n") 

print(train_data_bigram_cv)

(1829, 41006) 

(458, 41006) 

  (0, 37347)	1
  (0, 35638)	9
  (0, 35036)	5
  (0, 26826)	1
  (0, 38075)	1
  (0, 22562)	1
  (0, 5228)	2
  (0, 37876)	2
  (0, 7119)	2
  (0, 24675)	3
  (0, 22722)	2
  (0, 39767)	1
  (0, 9651)	3
  (0, 14656)	1
  (0, 7449)	1
  (0, 39475)	1
  (0, 18831)	6
  (0, 10297)	1
  (0, 29943)	3
  (0, 28240)	2
  (0, 23095)	1
  (0, 18313)	3
  (0, 27163)	2
  (0, 10007)	1
  (0, 33738)	16
  :	:
  (1826, 16885)	1
  (1827, 18831)	1
  (1827, 4887)	1
  (1827, 39808)	1
  (1827, 17933)	1
  (1827, 25276)	1
  (1827, 13528)	1
  (1827, 4958)	1
  (1827, 20432)	1
  (1827, 7401)	1
  (1827, 1099)	1
  (1827, 27219)	1
  (1827, 16051)	1
  (1827, 18845)	1
  (1827, 1100)	1
  (1827, 39925)	1
  (1827, 20433)	1
  (1827, 7402)	1
  (1827, 13529)	1
  (1827, 17971)	1
  (1827, 27220)	1
  (1827, 25369)	1
  (1828, 18831)	1
  (1828, 14581)	1
  (1828, 19049)	1


In [397]:
# Training a New Naive Bayes Classifier

# Define true labels from train set

x_train_bi_cv = train_data_bigram_cv
y_train_rev_series = full_train_df["rating"]
x_test_bi_cv = test_data_bigram_cv
y_test_rev_series = full_test_df["rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_bi_cv, y_train_rev_series)

# Predict the labels for the test data

bi_predictions = mnb_model.predict(x_test_bi_cv)
bi_predictions

array([1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 3, 1, 1, 4, 5, 5, 5, 1, 5, 5, 5,
       1, 5, 5, 1, 5, 5, 1, 5, 5, 5, 5, 5, 1, 5, 1, 5, 5, 5, 1, 5, 5, 5,
       5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 4, 5, 5, 1, 5, 5, 4, 1, 5,
       5, 5, 5, 1, 5, 4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 1, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5,
       1, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5,
       5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 4, 1, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,
       5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,

In [398]:
# Simpler Model Evaluation: Unigrams + Bigrams

true_bi_label_list = y_test_rev_series.to_list()
predict_bi_label_list = list(bi_predictions)

print ("Accuracy score for unigrams + bigrams: ", accuracy_score(true_bi_label_list, bi_predictions))
print ("Precision macro score for unigrams + bigrams: ", precision_score(true_bi_label_list, predict_bi_label_list, average= 'macro'))
print ("Recall macro score for unigrams + bigrams: ", recall_score(true_bi_label_list, predict_bi_label_list, average= 'macro'))
print ("F1 macro score for unigrams + bigrams: ", f1_score(true_bi_label_list, predict_bi_label_list, average= 'macro'))


Accuracy score for unigrams + bigrams:  0.6244541484716157
Precision macro score for unigrams + bigrams:  0.27141238135713275
Recall macro score for unigrams + bigrams:  0.28008542649496365
F1 macro score for unigrams + bigrams:  0.26332604636538254


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [399]:
# Evaluating Labels: 1,  Unigrams + Bigrams 

# Constructing Filter for Label == 1
bi_label_1 = 1 

bi_label_filter = [1 if label == bi_label_1 else 0 for label in true_bi_label_list]

true_bi_labels_for_label_1 = [true_bi_label_list[i] for i, mask in enumerate(bi_label_filter) if mask == 1]
predicted_bi_labels_for_lable_1 = [predict_bi_label_list[i] for i, mask in enumerate(bi_label_filter) if mask == 1]

# Accuracy Scoring for New List
accuracy_for_bi_label_1 = accuracy_score(true_bi_labels_for_label_1, predicted_bi_labels_for_lable_1)

# Precision Scoring for New List
prec_score_macro_bi_label_1 = precision_score(true_bi_labels_for_label_1, predicted_bi_labels_for_lable_1, average='macro')
prec_score_micro_bi_label_1 = precision_score(true_bi_labels_for_label_1, predicted_bi_labels_for_lable_1, average='micro')
prec_score_weighted_bi_label_1 = precision_score(true_bi_labels_for_label_1, predicted_bi_labels_for_lable_1, average='weighted')

# Recall Scoring for New List
rec_score_macro_bi_label_1 = recall_score(true_bi_labels_for_label_1, predicted_bi_labels_for_lable_1, average='macro')
rec_score_micro_bi_label_1 = recall_score(true_bi_labels_for_label_1, predicted_bi_labels_for_lable_1, average='micro')
rec_score_weighted_bi_label_1 = recall_score(true_bi_labels_for_label_1, predicted_bi_labels_for_lable_1, average='weighted')

#F1 Scoring for New List
f1_score_macro_bi_label_1 = f1_score(true_bi_labels_for_label_1, predicted_bi_labels_for_lable_1, average='macro')
f1_score_micro_bi_label_1 = f1_score(true_bi_labels_for_label_1, predicted_bi_labels_for_lable_1, average='micro')
f1_score_weighted_bi_label_1 = f1_score(true_bi_labels_for_label_1, predicted_bi_labels_for_lable_1, average='weighted')


# Reporting All 
print('Reporting Scores for Label 1 - unigrams + bigrams')
print()
print(f'Accuracy for label {bi_label_1}: {accuracy_for_bi_label_1:.2f}')
print()
print(f'macro precision score for label {bi_label_1} = {prec_score_macro_bi_label_1}')
print(f'micro precision score for label {bi_label_1} = {prec_score_micro_bi_label_1}')
print(f'weighted precision score for label {bi_label_1} = {prec_score_weighted_bi_label_1}')
print()
print(f'macro recall score for label {bi_label_1} = {rec_score_macro_bi_label_1}')
print(f'micro recall score for label {bi_label_1} = {rec_score_micro_bi_label_1}')
print(f'weighted recall score for label {bi_label_1} = {rec_score_weighted_bi_label_1}')
print()
print(f'macro F1 score for label {bi_label_1} = {f1_score_macro_bi_label_1}')
print(f'micro F1 score for label {bi_label_1} = {f1_score_micro_bi_label_1}')
print(f'weighted F1 score for label {bi_label_1} = {f1_score_weighted_bi_label_1}')


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Reporting Scores for Label 1 - unigrams + bigrams

Accuracy for label 1: 0.43

macro precision score for label 1 = 0.25
micro precision score for label 1 = 0.4260869565217391
weighted precision score for label 1 = 1.0

macro recall score for label 1 = 0.10652173913043478
micro recall score for label 1 = 0.4260869565217391
weighted recall score for label 1 = 0.4260869565217391

macro F1 score for label 1 = 0.14939024390243902
micro F1 score for label 1 = 0.4260869565217391
weighted F1 score for label 1 = 0.5975609756097561


In [411]:
# Evaluating Labels: 2,  Unigrams + Bigrams 

# Constructing Filter for Label == 2
bi_label_2 = 2 

bi_label_filter = [2 if label == bi_label_2 else 0 for label in true_bi_label_list]

true_bi_labels_for_label_2 = [true_bi_label_list[i] for i, mask in enumerate(bi_label_filter) if mask == 2]
predicted_bi_labels_for_lable_2 = [predict_bi_label_list[i] for i, mask in enumerate(bi_label_filter) if mask == 2]

# Accuracy Scoring for New List
accuracy_for_bi_label_2 = accuracy_score(true_bi_labels_for_label_2, predicted_bi_labels_for_lable_2)

# Precision Scoring for New List
prec_score_zero_bi_label_2 = precision_score(true_bi_labels_for_label_2, predicted_bi_labels_for_lable_2, average=None, zero_division=1)
prec_score_macro_bi_label_2 = precision_score(true_bi_labels_for_label_2, predicted_bi_labels_for_lable_2, average='macro')
prec_score_micro_bi_label_2 = precision_score(true_bi_labels_for_label_2, predicted_bi_labels_for_lable_2, average='micro')
prec_score_weighted_bi_label_2 = precision_score(true_bi_labels_for_label_2, predicted_bi_labels_for_lable_2, average='weighted')

# Recall Scoring for New List
rec_score_zero_bi_label_2 = recall_score(true_bi_labels_for_label_2, predicted_bi_labels_for_lable_2, average=None, zero_division= 1)
rec_score_macro_bi_label_2 = recall_score(true_bi_labels_for_label_2, predicted_bi_labels_for_lable_2, average='macro')
rec_score_micro_bi_label_2 = recall_score(true_bi_labels_for_label_2, predicted_bi_labels_for_lable_2, average='micro')
rec_score_weighted_bi_label_2 = recall_score(true_bi_labels_for_label_2, predicted_bi_labels_for_lable_2, average='weighted')

#F1 Scoring for New List
f1_score_zero_bi_label_2 = f1_score(true_bi_labels_for_label_2, predicted_bi_labels_for_lable_2, average=None, zero_division=1)
f1_score_macro_bi_label_2 = f1_score(true_bi_labels_for_label_2, predicted_bi_labels_for_lable_2, average='macro')
f1_score_micro_bi_label_2 = f1_score(true_bi_labels_for_label_2, predicted_bi_labels_for_lable_2, average='micro')
f1_score_weighted_bi_label_2 = f1_score(true_bi_labels_for_label_2, predicted_bi_labels_for_lable_2, average='weighted')


# Reporting All 
print('Reporting Scores for Label 2 - unigrams + bigrams')
print()
print(f'Accuracy for label {bi_label_2}: {accuracy_for_bi_label_2:.2f}')
print()
print(f'zero division precision score for label {bi_label_2} = {prec_score_zero_bi_label_2}')
print(f'macro precision score for label {bi_label_2} = {prec_score_macro_bi_label_2}')
print(f'micro precision score for label {bi_label_2} = {prec_score_micro_bi_label_2}')
print(f'weighted precision score for label {bi_label_2} = {prec_score_weighted_bi_label_2}')
print()
print(f'zero division recall score for label {bi_label_2} = {rec_score_zero_bi_label_2}')
print(f'macro recall score for label {bi_label_2} = {rec_score_macro_bi_label_2}')
print(f'micro recall score for label {bi_label_2} = {rec_score_micro_bi_label_2}')
print(f'weighted recall score for label {bi_label_2} = {rec_score_weighted_bi_label_2}')
print()
print(f'zero division F1 score for label {bi_label_2} = {f1_score_zero_bi_label_2}')
print(f'macro F1 score for label {bi_label_2} = {f1_score_macro_bi_label_2}')
print(f'micro F1 score for label {bi_label_2} = {f1_score_micro_bi_label_2}')
print(f'weighted F1 score for label {bi_label_2} = {f1_score_weighted_bi_label_2}')


Reporting Scores for Label 2 - unigrams + bigrams

Accuracy for label 2: 0.00

zero division precision score for label 2 = [0. 1. 0. 0. 0.]
macro precision score for label 2 = 0.0
micro precision score for label 2 = 0.0
weighted precision score for label 2 = 0.0

zero division recall score for label 2 = [1. 0. 1. 1. 1.]
macro recall score for label 2 = 0.0
micro recall score for label 2 = 0.0
weighted recall score for label 2 = 0.0

zero division F1 score for label 2 = [0. 0. 0. 0. 0.]
macro F1 score for label 2 = 0.0
micro F1 score for label 2 = 0.0
weighted F1 score for label 2 = 0.0


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonr

In [412]:
# Evaluating Labels: 3,  Unigrams + Bigrams 

# Constructing Filter for Label == 3
bi_label_3 = 3

bi_label_filter = [3 if label == bi_label_3 else 0 for label in true_bi_label_list]

true_bi_labels_for_label_3 = [true_bi_label_list[i] for i, mask in enumerate(bi_label_filter) if mask == 3]
predicted_bi_labels_for_lable_3 = [predict_bi_label_list[i] for i, mask in enumerate(bi_label_filter) if mask == 3]

# Accuracy Scoring for New List
accuracy_for_bi_label_3 = accuracy_score(true_bi_labels_for_label_3, predicted_bi_labels_for_lable_3)

# Precision Scoring for New List
prec_score_macro_bi_label_3 = precision_score(true_bi_labels_for_label_3, predicted_bi_labels_for_lable_3, average='macro')
prec_score_micro_bi_label_3 = precision_score(true_bi_labels_for_label_3, predicted_bi_labels_for_lable_3, average='micro')
prec_score_weighted_bi_label_3 = precision_score(true_bi_labels_for_label_3, predicted_bi_labels_for_lable_3, average='weighted')

# Recall Scoring for New List
rec_score_macro_bi_label_3 = recall_score(true_bi_labels_for_label_3, predicted_bi_labels_for_lable_3, average='macro')
rec_score_micro_bi_label_3 = recall_score(true_bi_labels_for_label_3, predicted_bi_labels_for_lable_3, average='micro')
rec_score_weighted_bi_label_3 = recall_score(true_bi_labels_for_label_3, predicted_bi_labels_for_lable_3, average='weighted')

#F1 Scoring for New List
f1_score_macro_bi_label_3 = f1_score(true_bi_labels_for_label_3, predicted_bi_labels_for_lable_3, average='macro')
f1_score_micro_bi_label_3 = f1_score(true_bi_labels_for_label_3, predicted_bi_labels_for_lable_3, average='micro')
f1_score_weighted_bi_label_3 = f1_score(true_bi_labels_for_label_3, predicted_bi_labels_for_lable_3, average='weighted')


# Reporting All 
print('Reporting Scores for Label 3 - unigrams + bigrams')
print()
print(f'Accuracy for label {bi_label_3}: {accuracy_for_bi_label_3:.2f}')
print()
print(f'macro precision score for label {bi_label_3} = {prec_score_macro_bi_label_3}')
print(f'micro precision score for label {bi_label_3} = {prec_score_micro_bi_label_3}')
print(f'weighted precision score for label {bi_label_3} = {prec_score_weighted_bi_label_3}')
print()
print(f'macro recall score for label {bi_label_3} = {rec_score_macro_bi_label_3}')
print(f'micro recall score for label {bi_label_3} = {rec_score_micro_bi_label_3}')
print(f'weighted recall score for label {bi_label_3} = {rec_score_weighted_bi_label_3}')
print()
print(f'macro F1 score for label {bi_label_3} = {f1_score_macro_bi_label_3}')
print(f'micro F1 score for label {bi_label_3} = {f1_score_micro_bi_label_3}')
print(f'weighted F1 score for label {bi_label_3} = {f1_score_weighted_bi_label_3}')


Reporting Scores for Label 3 - unigrams + bigrams

Accuracy for label 3: 0.00

macro precision score for label 3 = 0.0
micro precision score for label 3 = 0.0
weighted precision score for label 3 = 0.0

macro recall score for label 3 = 0.0
micro recall score for label 3 = 0.0
weighted recall score for label 3 = 0.0

macro F1 score for label 3 = 0.0
micro F1 score for label 3 = 0.0
weighted F1 score for label 3 = 0.0


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonr

In [402]:
# Evaluating Labels: 4,  Unigrams + Bigrams 

# Constructing Filter for Label == 4
bi_label_4 = 4

bi_label_filter = [4 if label == bi_label_4 else 0 for label in true_bi_label_list]

true_bi_labels_for_label_4 = [true_bi_label_list[i] for i, mask in enumerate(bi_label_filter) if mask == 4]
predicted_bi_labels_for_lable_4 = [predict_bi_label_list[i] for i, mask in enumerate(bi_label_filter) if mask == 4]

# Accuracy Scoring for New List
accuracy_for_bi_label_4 = accuracy_score(true_bi_labels_for_label_4, predicted_bi_labels_for_lable_4)

# Precision Scoring for New List
prec_score_macro_bi_label_4 = precision_score(true_bi_labels_for_label_4, predicted_bi_labels_for_lable_4, average='macro')
prec_score_micro_bi_label_4 = precision_score(true_bi_labels_for_label_4, predicted_bi_labels_for_lable_4, average='micro')
prec_score_weighted_bi_label_4 = precision_score(true_bi_labels_for_label_4, predicted_bi_labels_for_lable_4, average='weighted')

# Recall Scoring for New List
rec_score_macro_bi_label_4 = recall_score(true_bi_labels_for_label_4, predicted_bi_labels_for_lable_4, average='macro')
rec_score_micro_bi_label_4 = recall_score(true_bi_labels_for_label_4, predicted_bi_labels_for_lable_4, average='micro')
rec_score_weighted_bi_label_4 = recall_score(true_bi_labels_for_label_4, predicted_bi_labels_for_lable_4, average='weighted')

#F1 Scoring for New List
f1_score_macro_bi_label_4 = f1_score(true_bi_labels_for_label_4, predicted_bi_labels_for_lable_4, average='macro')
f1_score_micro_bi_label_4 = f1_score(true_bi_labels_for_label_4, predicted_bi_labels_for_lable_4, average='micro')
f1_score_weighted_bi_label_4 = f1_score(true_bi_labels_for_label_4, predicted_bi_labels_for_lable_4, average='weighted')


# Reporting All 
print('Reporting Scores for Label 4 - unigrams + bigrams')
print()
print(f'Accuracy for label {bi_label_4}: {accuracy_for_bi_label_4:.2f}')
print()
print(f'macro precision score for label {bi_label_4} = {prec_score_macro_bi_label_4}')
print(f'micro precision score for label {bi_label_4} = {prec_score_micro_bi_label_4}')
print(f'weighted precision score for label {bi_label_4} = {prec_score_weighted_bi_label_4}')
print()
print(f'macro recall score for label {bi_label_4} = {rec_score_macro_bi_label_4}')
print(f'micro recall score for label {bi_label_4} = {rec_score_micro_bi_label_4}')
print(f'weighted recall score for label {bi_label_4} = {rec_score_weighted_bi_label_4}')
print()
print(f'macro F1 score for label {bi_label_4} = {f1_score_macro_bi_label_4}')
print(f'micro F1 score for label {bi_label_4} = {f1_score_micro_bi_label_4}')
print(f'weighted F1 score for label {bi_label_4} = {f1_score_weighted_bi_label_4}')


Reporting Scores for Label 4 - unigrams + bigrams

Accuracy for label 4: 0.02

macro precision score for label 4 = 0.25
micro precision score for label 4 = 0.022727272727272728
weighted precision score for label 4 = 1.0

macro recall score for label 4 = 0.005681818181818182
micro recall score for label 4 = 0.022727272727272728
weighted recall score for label 4 = 0.022727272727272728

macro F1 score for label 4 = 0.011111111111111112
micro F1 score for label 4 = 0.022727272727272728
weighted F1 score for label 4 = 0.044444444444444446


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [403]:
# Evaluating Labels: 5,  Unigrams + Bigrams 

# Constructing Filter for Label == 5
bi_label_5 = 5

bi_label_filter = [5 if label == bi_label_5 else 0 for label in true_bi_label_list]

true_bi_labels_for_label_5 = [true_bi_label_list[i] for i, mask in enumerate(bi_label_filter) if mask == 5]
predicted_bi_labels_for_lable_5 = [predict_bi_label_list[i] for i, mask in enumerate(bi_label_filter) if mask == 5]

# Accuracy Scoring for New List
accuracy_for_bi_label_5 = accuracy_score(true_bi_labels_for_label_5, predicted_bi_labels_for_lable_5)

# Precision Scoring for New List
prec_score_macro_bi_label_5 = precision_score(true_bi_labels_for_label_5, predicted_bi_labels_for_lable_5, average='macro')
prec_score_micro_bi_label_5 = precision_score(true_bi_labels_for_label_5, predicted_bi_labels_for_lable_5, average='micro')
prec_score_weighted_bi_label_5 = precision_score(true_bi_labels_for_label_5, predicted_bi_labels_for_lable_5, average='weighted')

# Recall Scoring for New List
rec_score_macro_bi_label_5 = recall_score(true_bi_labels_for_label_5, predicted_bi_labels_for_lable_5, average='macro')
rec_score_micro_bi_label_5 = recall_score(true_bi_labels_for_label_5, predicted_bi_labels_for_lable_5, average='micro')
rec_score_weighted_bi_label_5 = recall_score(true_bi_labels_for_label_5, predicted_bi_labels_for_lable_5, average='weighted')

#F1 Scoring for New List
f1_score_macro_bi_label_5 = f1_score(true_bi_labels_for_label_5, predicted_bi_labels_for_lable_5, average='macro')
f1_score_micro_bi_label_5 = f1_score(true_bi_labels_for_label_5, predicted_bi_labels_for_lable_5, average='micro')
f1_score_weighted_bi_label_5 = f1_score(true_bi_labels_for_label_5, predicted_bi_labels_for_lable_5, average='weighted')


# Reporting All 
print('Reporting Scores for Label 5 - unigrams + bigrams')
print()
print(f'Accuracy for label {bi_label_5}: {accuracy_for_bi_label_4:.2f}')
print()
print(f'macro precision score for label {bi_label_5} = {prec_score_macro_bi_label_5}')
print(f'micro precision score for label {bi_label_5} = {prec_score_micro_bi_label_5}')
print(f'weighted precision score for label {bi_label_5} = {prec_score_weighted_bi_label_5}')
print()
print(f'macro recall score for label {bi_label_5} = {rec_score_macro_bi_label_5}')
print(f'micro recall score for label {bi_label_5} = {rec_score_micro_bi_label_5}')
print(f'weighted recall score for label {bi_label_5} = {rec_score_weighted_bi_label_5}')
print()
print(f'macro F1 score for label {bi_label_5} = {f1_score_macro_bi_label_5}')
print(f'micro F1 score for label {bi_label_5} = {f1_score_micro_bi_label_5}')
print(f'weighted F1 score for label {bi_label_5} = {f1_score_weighted_bi_label_5}')


Reporting Scores for Label 5 - unigrams + bigrams

Accuracy for label 5: 0.02

macro precision score for label 5 = 0.3333333333333333
micro precision score for label 5 = 0.9516129032258065
weighted precision score for label 5 = 1.0

macro recall score for label 5 = 0.3172043010752688
micro recall score for label 5 = 0.9516129032258065
weighted recall score for label 5 = 0.9516129032258065

macro F1 score for label 5 = 0.325068870523416
micro F1 score for label 5 = 0.9516129032258065
weighted F1 score for label 5 = 0.9752066115702479


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


6. Adding Tri-gram Features

In [404]:
# Extracting Tri-gram features 

# Building Model
train_text = full_train_df["review"]
test_text = full_test_df["review"]

# Setting n-gram range to uni-grams and bigrams
vectorizer = CountVectorizer(ngram_range = (1, 3))

# Creating training data representation
train_data_trigram_cv = vectorizer.fit_transform(train_text.values.astype('U'))
print(train_data_trigram_cv.shape,"\n") 

# Creating test data representation
test_data_trigram_cv = vectorizer.transform(test_text.values.astype('U'))
print(test_data_trigram_cv.shape,"\n") 

print(train_data_trigram_cv)

(1829, 97367) 

(458, 97367) 

  (0, 88181)	1
  (0, 83615)	9
  (0, 81998)	5
  (0, 63324)	1
  (0, 90083)	1
  (0, 53192)	1
  (0, 13161)	2
  (0, 89621)	2
  (0, 17504)	2
  (0, 58432)	3
  (0, 53540)	2
  (0, 94178)	1
  (0, 22813)	3
  (0, 33804)	1
  (0, 18396)	1
  (0, 93506)	1
  (0, 44026)	6
  (0, 24127)	1
  (0, 69557)	3
  (0, 66160)	2
  (0, 54558)	1
  (0, 42397)	3
  (0, 63984)	2
  (0, 23554)	1
  (0, 77753)	16
  :	:
  (1827, 18310)	1
  (1827, 2378)	1
  (1827, 64100)	1
  (1827, 37079)	1
  (1827, 44056)	1
  (1827, 2379)	1
  (1827, 94612)	1
  (1827, 48397)	1
  (1827, 18311)	1
  (1827, 31273)	1
  (1827, 41524)	1
  (1827, 64101)	1
  (1827, 60184)	1
  (1827, 44057)	1
  (1827, 2380)	1
  (1827, 94613)	1
  (1827, 48398)	1
  (1827, 18312)	1
  (1827, 12511)	1
  (1827, 31274)	1
  (1827, 41525)	1
  (1827, 64102)	1
  (1828, 44026)	1
  (1828, 33615)	1
  (1828, 44749)	1


In [405]:
# Training another New Naive Bayes Classifier

# Define true labels from train set

x_train_tri_cv = train_data_trigram_cv
y_train_rev_series = full_train_df["rating"]
x_test_tri_cv = test_data_trigram_cv
y_test_rev_series = full_test_df["rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_tri_cv, y_train_rev_series)

# Predict the labels for the test data

tri_predictions = mnb_model.predict(x_test_tri_cv)
tri_predictions

array([1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 3, 1, 1, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 1, 5, 5, 5, 1, 5, 5, 5,
       5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 4, 5, 5, 1, 5, 5, 4, 1, 5,
       5, 5, 5, 1, 5, 4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 1, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5,
       1, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 4, 1, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,
       5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,

In [406]:
# Simpler Model Evaluation: Unigrams + Bigrams + Trigrams

true_tri_label_list = y_test_rev_series.to_list()
predict_tri_label_list = list(tri_predictions)

print ("Accuracy score for unigrams + bigrams: ", accuracy_score(true_bi_label_list, bi_predictions))
print ("Precision macro score for unigrams + bigrams: ", precision_score(true_bi_label_list, predict_bi_label_list, average= 'macro'))
print ("Recall macro score for unigrams + bigrams: ", recall_score(true_bi_label_list, predict_bi_label_list, average= 'macro'))
print ("F1 macro score for unigrams + bigrams: ", f1_score(true_bi_label_list, predict_bi_label_list, average= 'macro'))


Accuracy score for unigrams + bigrams:  0.6244541484716157
Precision macro score for unigrams + bigrams:  0.27141238135713275
Recall macro score for unigrams + bigrams:  0.28008542649496365
F1 macro score for unigrams + bigrams:  0.26332604636538254


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
